# Empathy and Distress Inference with EmotionStack

In [1]:
from transformers import AutoConfig, AutoModelWithHeads
import torch
from utils import utils
from transformers import RobertaTokenizer
import transformers.adapters.composition as ac

In [2]:
""" load data """

train_data, val_data, test_data = utils.load_wassa_dataset()

In [3]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

config = AutoConfig.from_pretrained(
    "roberta-base",
    num_labels=1
)

model = AutoModelWithHeads.from_pretrained(
    "roberta-base",
    config=config,
)

/app/home/alahnala/miniconda3/envs/st/lib/python3.9/site-packages/transformers/adapters/models/roberta.py:250: FutureWarning: This class has been renamed to `RobertaAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(
/app/home/alahnala/miniconda3/envs/st/lib/python3.9/site-packages/transformers/adapters/models/roberta.py:228: FutureWarning: This class has been renamed to `RobertaAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a B

In [4]:
""" load adapters """
empathy_adapter_path = "./trained_adapters/EMP_emotion_stack"
distress_adapter_path = "./trained_adapters/DIS_emotion_stack"

empathy_adapter = model.load_adapter(empathy_adapter_path, load_as=empathy_adapter_path.split('/')[-1])
distress_adapter = model.load_adapter(distress_adapter_path, load_as=distress_adapter_path.split('/')[-1])

In [5]:
""" activate """
model.set_active_adapters(ac.Parallel(empathy_adapter, distress_adapter))

In [6]:
def pred_empathy_distress(sentence):
    tokens = tokenizer.tokenize(sentence)
    input_ids = torch.tensor(tokenizer.convert_tokens_to_ids(tokens))
    outputs = model(input_ids)

    emp = outputs[0].logits[0][0].tolist()
    dis = outputs[1].logits[0][0].tolist()
    return emp, dis


def predict_and_print(sentence):
    empathy_score, distress_score = pred_empathy_distress(sentence)
    
    print("Essay: ")
    print(sentence)
    print(f"Empathy: {empathy_score}\tDistress: {distress_score}")
    
    return empathy_score, distress_score

In [7]:
idx = 50

essay = val_data['essay'].values[idx]
empathy_score, distress_score = predict_and_print(essay)

print(f"\nActuals\nEmpathy: {val_data['empathy'].values[idx]}\tDistress: {val_data['distress'].values[idx]}")

Essay: 
I think that this is pretty sad.  I definitely think that there should be a lot more empathy for these people.  Say what you will, but to me it's just another example of the haves keeping the have nots as far down as possible.  I wonder how this article makes you feel and I am really curious to hear your opinions on the matter.
Empathy: 4.637167453765869	Distress: 3.3121418952941895

Actuals
Empathy: 6.0	Distress: 1.0
